In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import datetime
import pandas as pd
from seeq.addons.mps import _mps as mps


interactive_index = 0
if spy.jobs._common.running_in_executor():
    params = spy.jobs.pull()
else:
    params = spy.jobs.pull(interactive_index = interactive_index)

print("JOB PARAMETERS")
print(params)

workbook = params['workbook_name']
frequency = params['frequency']
user = params['user']
job_name = params['job_name']
paused = params['paused']

workbook_id = params['workbook_id']
worksheet_id = params['worksheet_id']

load_name = params['load_name']
known_cap = params['known_cap']
signal_pull_list = params['signal_pull_list']

time_frame = params['time_frame']
griding = params['griding']
mode = params['mode']
batch_cond = params['batch_cond_name']

return_top_x = int(params['return_top_x'])
similarity = params['similarity']
time_distort = params['time_distort']
found_cap_name = params['found_cap_name']
wb_id_scheduled = params['wb_id_scheduled']

In [ ]:
if not paused:
    try:
        if load_name == " ":

            desired_workbook, sheet_index = mps.gather_workbook_worksheet_meta_data(workbook_id, worksheet_id)
            items = desired_workbook[0].worksheets[sheet_index].display_items
            items_s = items[items.Type == 'Signal']
            items_s_ref = items_s[items_s['Name'].isin(signal_pull_list)]

            items_c = items[items.Type == 'Condition']
            items_c = items_c[items_c.Name == known_cap]
            data_pull_known = spy.pull(items_c, start=time_frame[0], end=time_frame[1], quiet=True, grid=griding)

        else:    
            items_s_ref, data_pull_known = mps.load_ref(load_name, 'References')

        og_start = time_frame[0]
        time_frame[0] = pd.Timestamp.now() - (time_frame[1]-time_frame[0])
        time_frame[1] = pd.Timestamp.now()

        data_pull, data_pull_c, sheet_index = mps.pull_mps_data(workbook_id,
                                                                worksheet_id,
                                                                signal_pull_list,
                                                                items_s_ref,
                                                                data_pull_known,
                                                                time_frame,
                                                                griding)
        
        if mode == "batch":

            desired_workbook = spy.workbooks.pull(spy.workbooks.search({'ID': workbook_id},
                                                                       quiet=True
                                                                       ),
                                                  include_referenced_workbooks=False,
                                                  quiet=True
                                                  )

            items = desired_workbook[0].worksheets[sheet_index].display_items
            items_c = items[items.Type == 'Condition']
            items_c = items_c[items_c.Name == batch_cond]
            batch_cond = spy.pull(items_c, start=time_frame[0], end=time_frame[1], quiet=True, grid=griding)
            batch_cond = batch_cond[batch_cond['Capsule Is Uncertain'] == False]
            
            for i in batch_cond.index:
                start_temp = batch_cond['Capsule Start'][i]
                if (start_temp.replace(tzinfo=None) < time_frame[0]):
                    batch_cond.drop(i, inplace=True)
            batch_cond.reset_index(drop=True, inplace=True)
            
            time_distort = 0.04
            
            Batch_sim_df = mps.seeq_mps_dtw_batch(batch_cond, data_pull, data_pull_c, data_pull_known,
                                                  True, time_distort)
            end = mps.push_mps_results_batch(Batch_sim_df, workbook_id, data_pull, found_cap_name, sheet_index,True, wb_id_scheduled)

        else:
            if mode == 'cont_1':
                min_idx_multivar = mps.seeq_mps_mass(data_pull,
                                                         data_pull_c,
                                                         data_pull_known,
                                                         similarity,
                                                         True,
                                                         True
                                                         )

            elif mode == 'cont_2':
                min_idx_multivar = mps.seeq_mps_dtw(data_pull,
                                                        data_pull_c,
                                                        data_pull_known,
                                                        similarity,
                                                        False,
                                                        True,
                                                        time_distort
                                                        )
            elif mode == 'cont_3':
                min_idx_multivar = mps.seeq_mps_mass(data_pull,
                                                         data_pull_c,
                                                         data_pull_known,
                                                         similarity,
                                                         False,
                                                         True
                                                         )

            else:
                min_idx_multivar = mps.seeq_mps_dtw(data_pull,
                                                        data_pull_c,
                                                        data_pull_known,
                                                        similarity,
                                                        True,
                                                        True,
                                                        time_distort
                                                        )

            end = mps.push_mps_results(return_top_x,
                                       min_idx_multivar,
                                       data_pull,
                                       workbook_id,
                                       found_cap_name,
                                       sheet_index,
                                       griding,
                                       True, wb_id_scheduled, og_start
                                       )
        
        job_status = 'SUCCESS'
        
    except Exception as e:
        print("There was an error when calling MPS...")
        print(f"workbook : {workbook}")
        print(f"name : {job_name}")
        print("EXCEPTION :")
        print(str(e))
        job_status = "FAILURE"
else:
    job_status = 'PAUSED'

try:
    status_df = pd.DataFrame({'Time': [pd.Timestamp.now()], f"{job_name} Status": [job_status]})
    status_df = status_df.set_index('Time')
    spy.push(status_df, workbook=workbook, quiet=True)
    print(f"Model Health Status ({job_status}) pushed to Seeq...")
except:
    print("Error pushing status point to Seeq...")    